# Pixeltable Complete Demo
## Unified Multimodal AI Framework

This notebook demonstrates **Pixeltable** - a revolutionary framework that unifies multimodal AI development.

In [ ]:
# Setup
import pixeltable as pxt
from pixeltable.functions import openai, huggingface
from pixeltable.iterators import DocumentSplitter
import os
from dotenv import load_dotenv
load_dotenv()
print("Setup complete!")

## Example 1: Auto-Calculated Profits

In [ ]:
pxt.create_dir("tutorial", ignore_errors=True)
films = pxt.create_table("tutorial.films", {"name": pxt.String, "revenue": pxt.Float, "budget": pxt.Float}, if_exists="replace")
films.insert([{"name": "Inside Out", "revenue": 800.5, "budget": 200.0}, {"name": "Toy Story", "revenue": 1073.4, "budget": 200.0}])
films.add_computed_column(profit=(films.revenue - films.budget), if_exists="replace")
print(films.select(films.name, films.profit).collect())

## Example 2: Image Analysis

In [ ]:
images = pxt.create_table("tutorial.images", {"input_image": pxt.Image}, if_exists="replace")
images.add_computed_column(vision_description=openai.vision(model="gpt-4o-mini", prompt="Describe this image", image=images.input_image))
print("Image analysis ready!")

## Example 3: Complete RAG System

In [ ]:
# Create documents table
docs = pxt.create_table("tutorial.docs", {"doc": pxt.Document}, if_exists="replace")
print("Documents table created")

In [ ]:
# Create chunks view
chunks = pxt.create_view("tutorial.doc_chunks", docs, iterator=DocumentSplitter.create(document=docs.doc, separators="sentence"), if_exists="replace")
print("Chunks view created")

In [ ]:
# Add embeddings
embed = huggingface.sentence_transformer(chunks.text, model_id="sentence-transformers/all-MiniLM-L6-v2")
chunks.add_computed_column(embedding=embed)
print("Embeddings added")

In [ ]:
# Insert sample document
import tempfile
sample = "Pixeltable simplifies AI development. It provides unified table interface for multimodal data."
with tempfile.NamedTemporaryFile(mode="w", suffix=".txt", delete=False) as f:
    f.write(sample)
    doc_path = f.name
docs.insert([{"doc": doc_path}])
print(f"Document inserted, chunks: {len(chunks.select().collect())}")

In [ ]:
# Create Q&A system
@pxt.query
def get_context(query_text: str, limit: int = 3):
    sim = chunks.embedding.similarity(query_text)
    return chunks.order_by(sim, asc=False).limit(limit).select(chunks.text)

qa = pxt.create_table("tutorial.qa", {"prompt": pxt.String}, if_exists="replace")
qa.add_computed_column(context=get_context(qa.prompt))
qa.add_computed_column(final_prompt=pxt.functions.string.format("CONTEXT:{0}\nQUESTION:{1}", qa.context, qa.prompt))
qa.add_computed_column(answer=openai.chat_completions(model="gpt-4o-mini", messages=[{"role":"user","content":qa.final_prompt}]).choices[0].message.content)
print("RAG system ready!")

In [ ]:
# Ask questions
qa.insert([{"prompt": "What is Pixeltable?"}])
result = qa.select(qa.answer).tail(1).collect()
print(f"Answer: {result[0]["answer"]}")

## Key Takeaways

- **Unified**: Everything in tables
- **Automatic**: Computed columns update automatically
- **Incremental**: Only changed data recomputed
- **Simple**: 30 lines vs 500+ for RAG